In [10]:
pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-w230neef
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-w230neef
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 454.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 2.9 MB/s eta 0:00:0000:0100:01
Using legacy 'setup.py install' for yake, since package 'wheel' is not installed.
  Running setup.py install for yake ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install rake-nltk

In [86]:
!python3 -m pip install pytextrank
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 6.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 KB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 KB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 KB 3.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.9/370

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [1]:
import os, re
from nltk.tokenize import word_tokenize

import nltk

from rake_nltk import Rake

import yake

import spacy, pytextrank

In [2]:
class Extractor:
    def __init__(self, language, max_tokens):
        self.language = language
        self.max_tokens = max_tokens
        pass

    def extract_terms(self, text):
        raise NotImplementedError("extract_terms method must be implemented in subclass")

    def extract_terms_with_span(self, text):
        terms = self.extract_terms(text)
        terms_with_span = self.find_term_span(text, terms)
        return terms_with_span

    def extract_terms_without_overlaps(self, text):
        terms_with_span = self.extract_terms_with_span(text)
        terms_without_overlaps = self.rmv_overlaps(terms_with_span)
        return terms_without_overlaps

    def postprocess_terms(self, terms):
        pass

    @staticmethod
    def find_term_span(text, terms):
        spans = []
        for t,score in terms:
          term = re.escape(t)
          patron = r'\b' + term + r'\b'
          coincidencias = re.finditer(patron, text, re.IGNORECASE)
          span = [(t, coincidencia.start(), coincidencia.end()-1, score) for coincidencia in coincidencias]
          spans.extend(span)
        return spans

    @staticmethod
    def rmv_overlaps(keywords):
      ent = [kw[0] for kw in keywords]
      pos = [kw[1:3] for kw in keywords]
      score = [kw[3:] for kw in keywords]
      updated_keywords = []
      repeated_words = []
      for i in range(len(ent)):
        overlap = False
        if pos[i] in repeated_words:
          overlap = True
        else:
          repeated_words.append(pos[i])
          for k in range(len(ent)):
            if (((pos[i][0] >= pos[k][0]) and (pos[i][1] < pos[k][1])) or ((pos[i][0] > pos[k][0]) and (pos[i][1] <= pos[k][1]))):
              overlap = True
        if (not overlap):
          kw = (ent[i],pos[i][0],pos[i][1]) + score[i]
          updated_keywords.append(kw)
      return updated_keywords

In [3]:
class RakeExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        self.stopwords = nltk.corpus.stopwords.words(language)
        self.extractor = Rake(stopwords=self.stopwords, language=language)

    def extract_terms(self, text):
        self.extractor.extract_keywords_from_text(text)
        terms = self.extractor.get_ranked_phrases_with_scores()
        terms = [(kw,score) for score,kw in terms if (len(word_tokenize(kw, language=self.language)) <= self.max_tokens)]
        return terms

In [4]:
class YakeExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        if language=='spanish':
            self.extractor = yake.KeywordExtractor() #aqui habria que poner top=70 por ejemplo
        else:
            raise ValueError("Expected spanish language. Other languages not recognised")

    def extract_terms(self, text):
        keywords = self.extractor.extract_keywords(text)
        terms = [(kw,score) for kw, score in keywords if (len(word_tokenize(kw)) <= self.max_tokens)]
        return terms

In [5]:
class TextRankExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        if language=='spanish':
            self.extractor = spacy.load("es_core_news_sm")
            self.extractor.add_pipe("textrank")
        else:
            raise ValueError("Expected spanish language. Other languages not recognised")

    def extract_terms(self, text):
        doc = self.extractor(text)
        terms = []
        for phrase in doc._.phrases:
          if (len(word_tokenize(phrase.text)) <= self.max_tokens):
            terms.append((phrase.text, phrase.rank))
        return terms

In [6]:
class TermExtractor:
    def __init__(self, extraction_methods="rake", language="spanish", max_tokens=3, join=False): #maybe fer que max_tokens s hagi de posar mes tard??
        self.extraction_methods = extraction_methods
        self.extractors = self.initialize_keyword_extractors(language, max_tokens)
        self.keywords = None
        self.join = join

    def __call__(self, text):
        terms = self.extract_terms(text, self.join)
        return terms
        
    def initialize_keyword_extractors(self, language, max_tokens):
        keyword_extractors = {} #esto esta hecho para mas de un extractor a la vez???
        
        if 'rake' in self.extraction_methods:
            keyword_extractors["rake"] = RakeExtractor(language, max_tokens)
        
        if 'yake' in self.extraction_methods:
            keyword_extractors["yake"] = YakeExtractor(language, max_tokens)
        
        if 'textrank' in self.extraction_methods:
            keyword_extractors["textrank"] = TextRankExtractor(language, max_tokens)

        if not keyword_extractors:
            raise ValueError("No extraction method called {}".format(self.extraction_methods))
        
        return keyword_extractors

    def extract_terms(self, text, join):
        try:
            all_terms = []
            for key, extractor in self.extractors.items():
                terms = extractor.extract_terms_without_overlaps(text)
                terms = [term + (key,) for term in terms]
                all_terms.extend(terms)
            if join:
                all_terms = list(self.extractors.values())[0].rmv_overlaps(all_terms)
            self.keywords = [Keyword(text=i[0], method=i[4], ini=i[1], fin=i[2], score=i[3]) for i in all_terms]
        except:
            raise AttributeError("A list of extractors must be provided")

In [19]:
class Keyword:
    def __init__(self, text, method, ini, fin, score):
        self.text = text
        self.method = method
        self.score = score
        self.span = [ini, fin]

    def __repr__(self):
        """
        A method to return a string representation of the Keyword object.

        Parameters:
        - None

        Returns:
        - A string representing the Keyword object.
        """
        return f"<Keyword(text='{self.text}', method='{self.method}', score='{self.score}', spans='{self.span}')>"

In [15]:
extractor = TermExtractor(extraction_methods=["textrank","rake"])

In [16]:
list(extractor.extractors.values())[0]

In [17]:
extractor("La importancia vital de lavarse con jabón las manos antes de operar es esencial", join=True)

In [18]:
a = [(kw.text,kw.method) for kw in extractor.keywords]
print(a)

[('operar', 'rake'), ('lavarse', 'rake'), ('jabón', 'rake'), ('esencial', 'rake'), ('La importancia vital', 'textrank'), ('las manos', 'textrank')]


In [ ]:
#estaria be fer el post processing abans de rmv_overlaps perq aixi les que passen a ser insignificants despres del postprocessing les eliminem

In [19]:
#Test de extracción completa directa
text = "Un varón de 32 años acude al Servicio de Urgencias por disminución reciente de visión en OD coincidiendo con la aparición de una lesión parduzca en dicho ojo. Entre los antecedentes oftalmológicos destaca un traumatismo penetrante en OD tres años antes que fue suturado en nuestro centro. A la exploración presenta una agudeza visual de 0,1 que mejora a 0,5 con estenopéico."

extractor1 = TermExtractor(extraction_methods="textrank", language="spanish", max_tokens=5)
print("Usando TextRank y un máximo de 5 tokens: \n",extractor1(text))

extractor2 = TermExtractor(extraction_methods=["rake", "yake"], language="spanish", max_tokens=2)
print("\nUsando Rake y Yake juntos y un máximo de 2 tokens: \n",extractor2(text))

extractor3 = TermExtractor(extraction_methods=["rake", "yake", "textrank"], language="spanish", max_tokens=3)
print("\nUsando Rake, Yake y TextRank juntos y un máximo de 3 tokens: \n",extractor3(text))

Usando TextRank y un máximo de 5 tokens: 
 None

Usando Rake y Yake juntos y un máximo de 2 tokens: 
 None

Usando Rake, Yake y TextRank juntos y un máximo de 3 tokens: 
 None


In [14]:
#Test de extracción por pasos: vemos que da lo mismo que el de antes
extractor = TermExtractor(extraction_methods="textrank", language="spanish", max_tokens=3)
terms = extractor.extractors["textrank"].extract_terms(text)
terms_with_span = extractor.extractors["textrank"].find_term_span(text,terms)
terms_without_overlaps = extractor.extractors["textrank"].rmv_overlaps(terms_with_span)
print(terms_without_overlaps)

[('dicho ojo', 148, 156), ('disminución reciente', 55, 74), ('OD', 89, 90), ('OD', 234, 235), ('estenopéico', 362, 372), ('visión', 79, 84), ('una lesión parduzca', 125, 143), ('Servicio de Urgencias', 29, 49), ('un traumatismo penetrante', 205, 229), ('nuestro centro', 273, 286), ('una agudeza visual', 315, 332), ('la aparición', 109, 120), ('32 años', 12, 18), ('tres años', 237, 245), ('los antecedentes oftalmológicos', 165, 195), ('Un varón', 0, 7), ('A la exploración', 289, 304), ('que', 253, 255), ('que', 341, 343)]


In [115]:
extractor.extractors["rake"].stopwords

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [17]:
extractor.extractors["rake"].extract_terms("La importancia vital de lavarse con jabón las manos antes de operar es indispensable")

['importancia vital', 'operar', 'manos', 'lavarse', 'jabón', 'indispensable']

In [ ]:
extractor_terminos = TermExtractor(sadasdasdsa)

In [ ]:
doc1_terms = extractor_terminos("textoasdasdasdsa")

In [ ]:
doc1_terms.zº

In [20]:
import sys, os, re
general_path = os.getcwd().split("BioTermCategorizer")[0]+"BioTermCategorizer/"
sys.path.append(general_path+'biotermcategorizer/')


In [36]:
from TermExtractor import TermExtractor

In [37]:
extractor = TermExtractor(extraction_methods=["textrank","rake"])

In [20]:
#sample texts
text1 = "Paciente varón de 35 años con tumoración en polo superior de teste derecho hallada de manera casual durante una autoexploración, motivo por el cual acude a consulta de urología donde se realiza exploración física, apreciando masa de 1cm aproximado de diámetro dependiente de epidídimo, y ecografía testicular, que se informa como lesión nodular sólida en cabeza de epidídimo derecho. Se realiza RMN. Confirmando masa nodular, siendo el tumor adenomatoide de epidídimo la primera posibilidad diagnóstica. Se decide, en los dos casos, resección quirúrgica de tumoración nodular en cola epidídimo derecho, sin realización de orquiectomía posterior. En ambos casos se realizó examen anátomopatológico de la pieza quirúrgica. Hallazgos histológicos macroscópicos: formación nodular de 1,5 cms (caso1) y 1,2 cms (caso 2) de consistencia firme, coloración blanquecina y bien delimitada. Microscópicamente se observa proliferación tumoral constituida por estructuras tubulares en las que la celularidad muestra núcleos redondeados y elongados sin atipia citológica y que ocasionalmente muestra citoplasmas vacuolados, todo ello compatible con tumor adenomatoide de epidídimo."
text2 = "Dos recién nacidos, varón y hembra de una misma madre y fallecidos a los 10 y 45 minutos de vida respectivamente a los que se les realizó examen necrópsico. El primero de los cadáveres, correspondiente a la hembra, fue remitido con el juicio clínico de insuficiencia respiratoria grave con sospecha de Síndrome de Potter con la constatación de oligoamnios severo; nació mediante cesárea urgente por presentación de nalgas y el test de Apgar fue 1/3/7; minutos más tarde falleció. El examen externo permitió observar una tonalidad subcianótica, facies triangular con hendiduras parpebrales mongoloides, micrognatia, raiz nasal ancha y occipucio prominente. El abdomen, globuloso, duro y ligeramente abollonado permitía la palpación de dos grandes masas ocupando ambas fosas renales y hemiabdomenes. A la apertura de cavidades destacaba la presencia de dos grandes masas renales de 10 x 8 x 5,5 cm y 12 x 8 x 6 cm con pesos de 190 y 235 gr respectivamente. Si bien se podía discernir la silueta renal, la superficie, abollonada, presentaba numerosas formaciones quísticas de contenido seroso; al corte dichos quistes mostraban un tamaño heterogéneo siendo mayores los situados a nivel cortical, dando al riñón un aspecto de esponja. Los pulmones derecho e izquierdo pesaban 17 y 15 gr (peso habitual del conjunto de 49 gr) mostrando una tonalidad rojiza uniforme; ambos se encontraban comprimidos como consecuencia de la elevación diafragmática condicionada por el gran tamaño de los riñones. El resto de los órganos no mostraba alteraciones macroscópicas significativas salvo las alteraciones posicionales derivadas de la compresión renal. En el segundo de los cadáveres, el correspondiente al varón, se observaron cambios morfológicos similares si bien el tamaño exhibido por los riñones era aún mayor, con pesos de 300 y 310 gr. El resto de las vísceras abdominales estaban comprimidas contra el diafragma. En ambos casos se realizó un estudio histológico detallado, centrado especialmente en los riñones en los que se demostraron múltiples quistes de distintos tamaños con morfología sacular a nivel cortical. Dichos quistes ocupaban la mayor parte del parénquima corticomedular si bien las zonas conservadas no mostraban alteraciones significativas salvo inmadurez focal. Dichos quistes estaban tapizados por un epitelio simple que variaba desde plano o cúbico. Los quistes medulares, de menor tamaño y más redondeados estaban tapizados por un epitelio de predominio cúbico. Después de las renales, las alteraciones más llamativas se encontraban en el hígado donde se observaron proliferación y dilatación, incluso quística, de los ductos biliares a nivel de los espacios porta. Con tales hallazgos se emitió en ambos casos el diagnóstico de enfermedad poliquística renal autosómica recesiva infantil."
text3 = "Paciente de 64 años, alérgico a penicilina y con recambio valvular aórtico por endocarditis que consultó por aparición de masa peneana de crecimiento progresivo en las últimas semanas. A la exploración física destacaba una formación excrecente y abigarrada en glande, que deformaba meato, con áreas ulceradas cubiertas de fibrina. Se palpaban adenopatías fijas y duras en ambas regiones inguinales. La radiografía de tórax y el TAC abdomino-pélvico confirmaron la presencia de adenopatías pulmonares e inguinales de gran tamaño. Con el diagnóstico de neoplasia de pene, se practicó penectomía parcial con margen de seguridad. La anatomía patológica demostró que se trataba de un sarcoma pleomórfico de pene con diferenciación osteosarcomatosa y márgenes libres de afectación. Se decidió tratamiento con dos líneas de quimioterapia consistente en adriamicina e ifosfamida pero no hubo respuesta. Ingresó de nuevo con recidiva local sangrante de gran tamaño y crecimiento rápido que provocaba obstrucción de meato con insuficiencia renal aguda. Se colocó sonda de cistostomía y se instauró tratamiento con sueroterapia, mejorando la función renal, pero con empeoramiento progresivo del estado general hasta que falleció a los 6 meses del diagnóstico."
text4 = "Mujer de 28 años sin antecedentes de interés que consultó por síndrome miccional con polaquiuria de predominio diurno y cierto grado de urgencia sin escapes urinario. El urocultivo resultó negativo por lo que se indicó tratamiento con anticolinérgicos. Ante la falta de respuesta al tratamiento, se realizó cistografía que fué normal y ecografía renovesical en la que se apreciaban imágenes quísticas parapiélicas, algunas de ellas con tabiques internos y vejiga sin lesiones. Con el fin de precisar la naturaleza de dichos quistes se solicitó TAC-abdominal, que informaba de gran quiste parapiélico en riñón derecho sin repercusión sobre la vía y una masa hipodensa suprarrenal derecha. La resonancia magnética demostró normalidad de la glándula suprarrenal y una lesión quística lobulada conteniendo numerosos septos en su interior que rodeaba al riñón derecho; en la celda renal izquierda existía una lesión de características similares pero de menor tamaño. Los hallazgos eran compatibles con linfangioma renal bilateral. Tras tres años de seguimiento la paciente continua con leve síntomatologia miccional en tratamiento, pero no ha presentado síntomas derivados de su lesión renal."
text5 = "Varón de 68 años, con antecedentes de hemorragia digestiva alta por aspirina y accidente isquémico transitorio a tratamiento crónico con trifusal (300 mg cada 12 horas), que acudió al Servicio de Urgencias del Hospital San Agustín (Avilés, Asturias), en mayo de 2006, por dolor en hemiabdomen izquierdo, intenso, continuo, de instauración súbita y acompañado de cortejo vegetativo. A la exploración presentaba una tensión arterial de 210/120 mm Hg, una frecuencia cardíaca de 80 por minuto, y dolor en fosa ilíaca izquierda, acentuado con la palpación. El hemograma (hemoglobina: 13 g/dL, plaquetas: 249.000), el estudio de coagulación, la bioquímica elemental de sangre, el sistemático de orina, el electrocardiograma y la radiografía simple de tórax eran normales. En la tomografía computarizada de abdomen se objetivó un extenso hematoma, de 12 cm de diámetro máximo, en la celda renal izquierda, sin líquido libre intraperitoneal; la suprarrenal izquierda quedaba englobada y no se podía identificar, y la derecha no presentaba alteraciones. La HTA no se llegó a controlar en Urgencias, a pesar del tratamiento con analgésicos, con antagonistas del calcio y con inhibidores de la enzima convertidora de la angiotensina II, por lo que el paciente, que mantenía cifras tensionales de 240/160 mm Hg, pasó a la Unidad de Cuidados Intensivos, para tratamiento intravenoso con nitroprusiato y labetalol. En las 24 horas siguientes se yuguló la crisis hipertensiva, y se comprobó que la hemoglobina y el hematocrito permanecían estables. Con la sospecha diagnóstica de rotura no traumática de un feocromocitoma pre-existente, se determinaron metanefrinas plasmáticas, que fueron normales, y catecolaminas y metanefrinas urinarias. En la orina de 24 horas del día siguiente al ingreso se obtuvieron los siguientes resultados: adrenalina: 65,1 mcg (valores normales -VN: 1,7-22,5), noradrenalina: 151,1 mcg (VN: 12,1-85,5), metanefrina: 853,5 mcg (VN: 74-297) y normetanefrina: 1396,6 mcg (VN: 105-354). A los 10 días, todavía ingresado el paciente, las cifras urinarias se habían normalizado por completo de modo espontáneo. Respecto al hematoma, en julio de 2006 no se había reabsorbido y persistía una imagen pseudoquística en la zona suprarrenal izquierda. En septiembre de 2006 se practicó una suprarrenalectomía unilateral, y el estudio histológico mostró una masa encapsulada de 6 x 5 cm, con necrosis hemorrágica extensa y algunas células corticales sin atipias."

In [46]:
%%time
extractor = TermExtractor(extraction_methods=["rake"])
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

55
141
67
52
152
CPU times: user 160 ms, sys: 1.38 ms, total: 161 ms
Wall time: 159 ms


In [42]:
%%time
extractor = TermExtractor(extraction_methods=["yake"])
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

15
35
19
20
14
CPU times: user 545 ms, sys: 5.49 ms, total: 551 ms
Wall time: 589 ms


In [43]:
%%time
extractor = TermExtractor(extraction_methods=["textrank"])
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

38
95
52
53
85
CPU times: user 1.04 s, sys: 88.3 ms, total: 1.12 s
Wall time: 1.95 s


In [21]:
%%time
extractor = TermExtractor(extraction_methods=["rake","yake","textrank"], join=True)
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

68
173
84
76
164
CPU times: user 1.76 s, sys: 97.7 ms, total: 1.86 s
Wall time: 2.46 s


In [22]:
extractor(text1)

In [26]:
a = extractor.keywords[0]
print(a)

<Keyword(text='realizó examen anátomopatológico', method='rake', score='9.0', spans = '[664, 695]')>
